##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [1]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
import networkx as nx
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False
from __future__ import print_function

In [2]:
import yaml

In [3]:
import datetime
def dateheure():
    return datetime.datetime.utcnow().strftime('%y%m%d%H%M')

In [4]:
saut="\n"

###Préparation des matrices de traits

In [5]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')

In [6]:
sampleNumber="03-X"
genDigraphe=False
genGraphe=False
samplePrefix="MGC-150815-extend-%s-paradigmes"%sampleNumber
sampleFile=samplePrefix+".csv"
goldFile="MGC-150815-total-1508150555-paradigmes.csv"
analysisPrefix="MGC-150815-extend-%s"%sampleNumber
logfile_name=analysisPrefix+samplePrefix+".log"
logfile = codecs.open("2015-Data/"+logfile_name,mode='w',encoding="utf8")

###Préparation des cases du paradigme

In [7]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires

#Préparation du calcul des analogies

###Calcul de la différence entre deux formes

In [8]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

###Accumulation des paires appartenant à un patron

In [9]:
def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])

###Transformation d'un patron en RegExp

In [10]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

###Substitution de sortie 
???

In [11]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

#Classe pour la gestion des patrons, des classes et des transformations

In [12]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print ("le numéro de forme n'est pas dans [1,2]",n,file=logfile)

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                if debug:
                    print (forme, file=logfile)
                    print ("pas de classe",idClasseForme, file=logfile)
                    print ("%.2f par forme de sortie" % (float(1)/len(classeForme)), file=logfile)
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print (forme, file=logfile) 
            print ("pas de patron", file=logfile)
        return sortieForme
        

##Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [13]:
def OLDrapports(paradigme):
    (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    if len(paradigme)>0:
#        for index, row in paradigme.iterrows():
#            patrons.ajouterFormes(row[0],row[1],diff(row[0],row[1]))
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
    return patrons.calculerGM()

In [14]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

###Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [15]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


In [16]:
paradigmes=pd.read_csv("2015-Data/"+sampleFile,sep=";",encoding="utf8")
del paradigmes[u"Unnamed: 0"]
#paradigmes

In [17]:
phonologicalMap=analysisPrefix[-2:]
if debug: print(phonologicalMap)
neutralisationsNORD=(u"6û",u"9ê")
neutralisationsSUD=(u"e2o",u"E9O")
if phonologicalMap=="-N":
    neutralisations=neutralisationsNORD
elif phonologicalMap=="-S":
    neutralisations=neutralisationsSUD
else:
    neutralisations=(u"",u"")
    phonologicalMap=("-X")
bdlexiqueIn = unicode(u"èò"+neutralisations[0])
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO"+neutralisations[1])
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [18]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    else:
        result=chaine
    return result

In [19]:
GOLD=pd.read_csv("2015-Data/"+goldFile,sep=";",encoding="utf8")
del GOLD[u"Unnamed: 0"]
goldCases=GOLD.columns.tolist()
goldCases.remove(u"lexeme")
for case in goldCases:
    GOLD[case]=GOLD[case].apply(lambda x: recoder(x))
    
paradigmesGOLD=GOLD[paradigmes.columns.values.tolist()]
#paradigmesGOLD

- sampleCases pour la liste des cases effectivement représentées dans le corpus de départ 

In [20]:
sampleCases=paradigmes.columns.values.tolist()
sampleCases.remove(u"lexeme")

In [21]:
paradigmes.stack().value_counts(dropna=True).sum()

49485

In [22]:
with open("2015-Data/"+analysisPrefix+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

###Comparer les cases analysées avec l'ensemble de toutes les cases

In [23]:
analyseCases=list(set([case for (case,autre) in resultatsLecture.keys()]))
if sorted(analyseCases)!=sorted(casesTotales):
    print ("Attention l'analyse ne comprend pas toutes les cases")

In [24]:
class paradigmeDistribution:
    '''
    Gestion des distributions dans les cases du paradigme
    '''

    def __init__(self,lexeme):
        self.lexeme=lexeme
        self.formes={i:{} for i in analyseCases}

    def ajouterFormes(self,case,formes,coef=1.0):
        for forme in formes:
            if not forme in self.formes[case]:
                self.formes[case][forme]=0
            self.formes[case][forme]+=formes[forme]*coef
            
    def normaliserDistributions(self,caseListe=analyseCases):
        normalesDistributions={i:{} for i in caseListe}
        for case in caseListe:
            total=0
            for element in self.formes[case]:
                total+=self.formes[case][element]
            for element in self.formes[case]:
                normalesDistributions[case][element]=float(self.formes[case][element])/total
        return normalesDistributions
        

In [25]:
def generateForms(lexeme):
    candidats=paradigmeDistribution(lexeme)
    casesSamples=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
    casesSamples.remove("lexeme")
    for caseDepart in casesSamples:
        formeDepart=paradigmes[paradigmes["lexeme"]==lexeme][caseDepart].iloc[0]
        if debug: print (caseDepart,formeDepart, file=logfile)
#        if formeDepart!="nan":
        for case in analyseCases:
            if debug: print (case, file=logfile)
            if not isinstance(resultatsLecture[(caseDepart, case)],str):
                if "," in formeDepart:
                    formesDepart=formeDepart.split(",")
                    coef=1.0/len(formesDepart)
                    for element in formesDepart:
                        candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(element),coef)
                else:
                    candidats.ajouterFormes(case,resultatsLecture[(caseDepart, case)].sortirForme(formeDepart))
            else: 
                if debug: print ("str", resultatsLecture[(caseDepart, case)], file=logfile)
    return candidats

In [26]:
def ajouterPoint(lexeme,forme,case,digraphe,graphe):
    pointName="%s-%s-%s"%(lexeme,forme,case)
#    if not pointName in digraphe.nodes():
    tam=case[:2]
    if tam=="in": tam="inf"
    digraphe.add_node(pointName, tam='"%s"'%tam)
    graphe.add_node(pointName, tam='"%s"'%tam)
    return pointName

def ajouterFleche(pointDepart,pointSortie,coef,digraphe,graphe):
    digraphe.add_edge(pointDepart,pointSortie,weight=float(coef))
    if digraphe.has_edge(pointSortie,pointDepart):
        coefGraphe=float(digraphe.edge[pointSortie][pointDepart]["weight"]+coef)/2
        graphe.add_edge(pointDepart,pointSortie,weight=coefGraphe)

In [27]:
def generateParadigms(generation1,genDigraphe=True):
    lexeme=generation1.lexeme
    distributionInitiale=generation1.normaliserDistributions()
    candidats=paradigmeDistribution(lexeme)
    digraphe=nx.DiGraph()
    graphe=nx.Graph()    
    for caseDepart in analyseCases:
        for formeDepart in distributionInitiale[caseDepart]:
            if formeDepart:
                pointDepart=ajouterPoint(lexeme,formeDepart,caseDepart,digraphe,graphe)
                coefDepart=distributionInitiale[caseDepart][formeDepart]
                if debug: print (caseDepart,formeDepart, file=logfile)
                for caseSortie in analyseCases:
                    distributionSortieBrute=resultatsLecture[(caseDepart, caseSortie)].sortirForme(formeDepart)
                    if distributionSortieBrute:
                        if not genDigraphe:
#                            print ("brute",distributionSortieBrute)
                            distributionSortie={f:distributionSortieBrute[f] for f in distributionSortieBrute if f in distributionInitiale[caseSortie]}
                        else:
                            distributionSortie=distributionSortieBrute
#                        print ("filtre",distributionSortie)
#                        print (distributionInitiale[caseSortie])
                        if debug: print (caseSortie,distributionSortie,distributionInitiale[caseDepart], file=logfile)
                        candidats.ajouterFormes(caseSortie,distributionSortie,distributionInitiale[caseDepart][formeDepart])
                        for formeSortie in distributionSortie:
                            pointSortie=ajouterPoint(lexeme,formeSortie,caseSortie,digraphe,graphe)
                            coefSortie=distributionSortie[formeSortie]
                            ajouterFleche(pointDepart,pointSortie,float(coefDepart*coefSortie),digraphe,graphe)
    return (candidats,digraphe,graphe)

In [28]:
def generate(lexeme,genDigraphe=True):
#    print (lexeme,end=", ")
    generation1=generateForms(lexeme)
#    print ("génération 2",end=", ")
    (generation2,lexDigraphe,lexGraphe)=generateParadigms(generation1,genDigraphe)
    lexCliques=list(nx.algorithms.clique.find_cliques(lexGraphe))
#    print (lexCliques)
#    print ("génération 3")
    return (generation2,lexDigraphe,lexGraphe,lexCliques)

In [29]:
len(paradigmes.dropna(thresh=2)["lexeme"])

4547

In [30]:
paradigmes.dropna(thresh=2).count().sum()-paradigmes.dropna(thresh=2)["lexeme"].count()

44938

####Calculer le score de la clique

In [ ]:
def cliqueScore(clique,graph):
    score=0
    for (depart,arrivee) in it.combinations_with_replacement(clique,2):
        score+=graph[depart][arrivee]["weight"]
    return score

##Génération des formes

In [ ]:
%%time
debug=False
#listeTest=[u"manger",u"boire",u"dormir",u"aller",u"neiger"]
listeTest=paradigmes.dropna(thresh=2)["lexeme"].values.tolist()
#listeTest=[u"abasourdir",u"évacuer"]
nbVerbes=len(listeTest)
print (nbVerbes)
globDigraphe=nx.DiGraph()
globGraphe=nx.Graph()
cliques=[]
cliquesScores={}
cliquesListes={}
numClique=0
for i,element in enumerate(listeTest):
    if (i%100)==0: print (i, dateheure()[-4:], int(100*float(i)/nbVerbes), end=", ")
    result=generate(element,genDigraphe)
    (generation,lexDigraphe,lexGraphe,lexCliques)= result
#    print (generation,lexDigraphe,lexGraphe,lexCliques)
    if genDigraphe:
        globDigraphe=nx.union(globDigraphe,lexDigraphe)
    if genGraphe:
        globGraphe=nx.union(globGraphe,lexGraphe)
    cliques.extend(lexCliques)
    for clique in lexCliques:
        cliquesScores[numClique]=cliqueScore(clique,lexGraphe)
        cliquesListes[numClique]=clique
        numClique+=1

In [ ]:
%%time
versionStamp=dateheure()
if genDigraphe: 
    nx.write_dot(globDigraphe,u"2015-Data/digraphe-%s.dot"%versionStamp)
if genGraphe:
    nx.write_dot(globGraphe,u"2015-Data/graphe-%s.dot"%versionStamp)

In [ ]:
print (len(cliques))

In [ ]:
longueurCliques={}
for clique in cliques:
    longueur=len(clique)
    if not longueur in longueurCliques:
        longueurCliques[longueur]=1
    else:
        longueurCliques[longueur]+=1
longueurCliques

###Faire la liste des cases lexicalisées de l'échantillon

In [ ]:
nbFormesLexicales={}
casesLexicales={element:paradigmes[paradigmes["lexeme"]==element].columns[paradigmes[paradigmes["lexeme"]==element].notnull().iloc[0]].tolist() for element in listeTest}
for element in casesLexicales:
    casesLexicales[element].remove("lexeme")
    nbFormesLexicales[element]=len(casesLexicales[element])

##Comparer la sortie des cliques avec le paradigme de départ

In [ ]:
%%time
seuilClique=1
#paradigmesCLIQUES=pd.DataFrame(columns=paradigmes.columns.values.tolist())
paradigmesListe=[]
#for n,clique in enumerate(sorted(cliques,key=lambda x: len(x),reverse=True)):
for n,clique in enumerate(cliques):
#    if seuilClique==0:
#        seuilClique=len(clique)-15
    if len(clique)>seuilClique:
        paradigmeClique={}
        sampleOK=True
#        print (n, len(clique))
        point=clique[0].split("-")
        lPoint=len(point)
#        print (point,"-".join(point[0:len(point)-2]))
        if lPoint==3:
            lexeme=point[0]
        else:
            lexeme="-".join(point[0:len(point)-2])
        paradigmeClique["lexeme"]=lexeme
        if n%5000==0: print (n,int(100*float(n)/len(cliques)),end=", ")
#        casesLexeme=paradigmes[paradigmes["lexeme"]==lexeme].notnull().columns.tolist()
#        casesLexeme=paradigmes[paradigmes["lexeme"]==lexeme].columns[paradigmes[paradigmes["lexeme"]==lexeme].notnull().iloc[0]].tolist()
#        casesLexeme.remove("lexeme")
        casesLexeme=casesLexicales[lexeme]
        nbInitial=len(casesLexeme)
        if casesLexeme and len(casesLexeme)<=len(clique):
            for element in casesLexeme:
#                print (lexeme,element, paradigmes[paradigmes["lexeme"]==lexeme][element])
                champForme=paradigmes[paradigmes["lexeme"]==lexeme][element].iloc[0]
                if ","  in champForme:
                    formes=champForme.split(",")
                    nbInitial+=len(formes)-1
                    okFormes=False
                    for forme in formes:
                        pointCase=u"%s-%s-%s"% (lexeme,forme,element)
#                        if debug: print (pointCase, clique)
                        if pointCase in clique:
                            okFormes=True
                            print ("point",pointCase)
#                            print ("clique",clique)
                            print ("gold",GOLD[GOLD["lexeme"]==lexeme][element])
                    if okFormes:
                        sampleOK=True
                    else:
                        sampleOK=False
                        break
                else:
                    forme=champForme
                    pointCase=u"%s-%s-%s"% (lexeme,forme,element)
                    if debug: print (pointCase, clique)
                    if not pointCase in clique:
                        sampleOK=False
#                        print ("point",pointCase)
#                        print ("clique",clique)
#                        print ("gold",GOLD[GOLD["lexeme"]==lexeme][element])
                        break
        else:
            sampleOK=False
        if sampleOK:
            for element in clique:
                elements=element.split("-")
                forme=elements[-2]
                taminfo=elements[-1]
#                try:
#                 (lexeme,forme,taminfo)=element.split("-")
#                except ValueError:
#                    print (element)
                paradigmeClique[taminfo]=forme
#                if taminfo in paradigmes.columns:
#                    if not paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].isnull().item():
#                        if paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item()!=forme:
#                            sampleOK=False
#                            print (paradigmes[(paradigmes["lexeme"]==lexeme)][taminfo].item(),forme,taminfo,end=", ")
#            if n<100: print (clique==cliquesListes[n])
            paradigmeClique["score"]=cliquesScores[n]
            paradigmeClique["ajouts"]=len(clique)-nbInitial
#            print (cliqueScore(clique),clique)
            paradigmesListe.append(paradigmeClique)
        else:
            if debug:
                print ()
                print (lexeme,clique)
                print ()
#    else:
#        print ("break")
#        print (n,lexeme,clique)
#        break

paradigmesCLIQUES=pd.DataFrame(paradigmesListe,columns=GOLD.columns.values.tolist()+[u"score",u"ajouts"])
#print (seuilClique)

In [ ]:
paradigmeSILVER=paradigmesCLIQUES[paradigmesCLIQUES["score"]==paradigmesCLIQUES.groupby(["lexeme"])["score"].transform(max)].reset_index()
del paradigmeSILVER["index"]

In [ ]:
paradigmeSILVER.to_csv(path_or_buf="2015-Data/"+analysisPrefix+'-Silver.csv',encoding="utf8",sep=";")
GOLD.to_csv(path_or_buf="2015-Data/"+analysisPrefix+'-Gold.csv',encoding="utf8",sep=";")

In [ ]:
import sys
sys.exit()

In [ ]:
def compareGOLD(row):
    global n
#    if row["lexeme"]=="dodo": 
#        debug=True
#    else:
#        debug=False
    if n%250==0: print (n,end=", ")
    n+=1
    ligne={k:row[k] for k in row.keys() if not k in ["lexeme","score","ajouts"]}
    lexeme=row["lexeme"]
    score=row["score"]
    ajouts=row["ajouts"]
    if not lexeme in conjugaisons:
        conjugaisons[lexeme]={"diff":[],"miss":[],"over":[],"overmiss":[],"ok":[]}
    if not lexeme in cliquesTypes:
        cliquesTypes[lexeme]={"diff":[],"miss":[],"over":[],"overmiss":[],"ok":[]}
    identiques=[]
    differents=[]
    missing=[]
    over=[]
    for case in analyseCases:
        if debug:    
            print (lexeme,case)
            print (GOLD[GOLD["lexeme"]==lexeme][case].item())
            print ((GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]))
        if not case in row:
            if debug: print ("not case")
            missing.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print ("absence",case)
        elif case in row[row.isnull()] and not GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
            if debug: print ("not case")
            missing.append(u"%s : Ø ≠ %s" % (case,GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print ("CLIQUES +NaN",case)
        elif not case in row[row.isnull()] and GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]:
            if debug: print ("not case")
            over.append(u"%s : %s ≠ Ø" % (case,row[case]))
            lexical=False
            if debug: print ("GOLD -NaN",case)            
        elif (row[case]==GOLD[GOLD["lexeme"]==lexeme][case].item()):
            if debug: print ("egal")
            identiques.append(u"%s : %s" % (case,row[case]))
            if debug: print ("valeurs id")
        elif (case in row[row.isnull()]) and (GOLD[GOLD["lexeme"]==lexeme][case].isnull().iloc[0]):
            if debug: print ("NaNs")
            identiques.append(u"%s : %s" % (case,u"ØØØ"))
            if debug: print ("deux NaN")
        else:
            if debug: print ("else")
            differents.append(u"%s : %s ≠ %s" % (case, row[case],GOLD[GOLD["lexeme"]==lexeme][case].item()))
            lexical=False
            if debug: print (u"différence",case)
    if differents:
        conjugaisons[lexeme]["diff"].append(differents)
        cliquesTypes[lexeme]["diff"].append((ligne,score,ajouts))
        if debug: print ("DIFF",len(differents),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: print (", ".join(differents))
    if missing and over:
        conjugaisons[lexeme]["overmiss"].append(over+missing)
        cliquesTypes[lexeme]["overmiss"].append((ligne,score,ajouts))
        if debug: print ("OVERMISS", len(over), len(missing),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(missing))
            print (", ".join(over))
    elif missing:
        conjugaisons[lexeme]["miss"].append(missing)
        cliquesTypes[lexeme]["miss"].append((ligne,score,ajouts))
        if debug: print ("MISS", len(missing),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(missing))
    elif over:
        conjugaisons[lexeme]["over"].append(over)
        cliquesTypes[lexeme]["over"].append((ligne,score,ajouts))
        if debug: print ("OVER", len(over),"=>", len(identiques), len(identiques)-nombreElements[lexeme], end=saut)
        if debug1: 
            print (", ".join(over))
    if identiques:
        conjugaisons[lexeme]["ok"].append(identiques)
        cliquesTypes[lexeme]["ok"].append((ligne,score,ajouts))
        if debug: print ("OK", len(identiques), len(identiques)-nombreElements[lexeme],end=saut)
        if debug1: print (", ".join(identiques))
#    print ()
    return lexical

In [ ]:
%%time
debug=False
debug1=False
saut="\n"
n=0
conjugaisons={}
cliquesTypes={}
print (", ".join(sampleCases))
#paradigmesCLIQUES.apply(compareGOLD,axis=1)
paradigmeSILVER.apply(compareGOLD,axis=1)

with codecs.open("2015-Data/"+samplePrefix+".yaml", 'w', encoding='utf8') as outfile:
    outfile.write(yaml.dump(conjugaisons, default_flow_style=True, encoding='utf8'))

In [ ]:
yaml.safe_dump(conjugaisons, file("2015-Data/"+samplePrefix+dateheure()+".yaml",'w'), encoding='utf-8', allow_unicode=True)
yaml.safe_dump(cliquesTypes, file("2015-Data/"+samplePrefix+"cliques-"+dateheure()+".yaml",'w'), encoding='utf-8', allow_unicode=True)

pb=[i for i in conjugaisons if conjugaisons[i]["diff"]]
print (len(pb))
for element in pb:
    print (element)
    print ("DIFF",conjugaisons[element]["diff"])
    print ("MISS",conjugaisons[element]["miss"],end="\n\n")

good=[i for i in conjugaisons if not conjugaisons[i]["diff"]]
for lexeme in good:
    print (lexeme)
    for ident in conjugaisons[lexeme]:
        for element in conjugaisons[lexeme][ident]:
            print (ident)
            print (", ".join(element))
    print ()

In [ ]:
precisions={"bon":0,"mauvais":0}
for verbe in cliquesTypes:
    if debug: print (verbe)
    for cle in cliquesTypes[verbe]:
        if debug: print (cle)
        for element in cliquesTypes[verbe][cle]:
            if debug: print (element[1],element[2])
            if cle in ["ok"]:
                precisions["bon"]+=element[2]
            elif cle in ["diff"]:
                precisions["mauvais"]+=element[2]
precisions

In [ ]:
float(precisions["bon"])/(precisions["bon"]+precisions["mauvais"])

In [ ]:
basesMesures={"total":0,"bon":0,"mauvais":0}
for verbe in conjugaisons:
    for cat in conjugaisons[verbe]:
        if conjugaisons[verbe][cat]:
#            print (verbe,cat,len(conjugaisons[verbe][cat][0]))
            if cat=="miss":
                basesMesures["total"]+=len(conjugaisons[verbe][cat][0])
            elif cat=="ok":
                basesMesures["bon"]+=len(conjugaisons[verbe][cat][0])
                basesMesures["total"]+=len(conjugaisons[verbe][cat][0])
            elif cat=="diff":
                basesMesures["mauvais"]+=len(conjugaisons[verbe][cat][0])
                basesMesures["total"]+=len(conjugaisons[verbe][cat][0])
            elif cat=="overmiss":
                basesMesures["mauvais"]+=len(conjugaisons[verbe][cat][0])
                basesMesures["total"]+=len(conjugaisons[verbe][cat][0])
basesMesures

In [ ]:
precisionManip=float(basesMesures["bon"])/(basesMesures["bon"]+basesMesures["mauvais"])
precisionManip

In [ ]:
rappelManip=float(basesMesures["bon"])/(basesMesures["total"])
rappelManip